### <p style="text-align: right;"> &#9989; **Group 1 (GoGreen)** </p>
#### <p style="text-align: right;"> &#9989; Beth, Zhongjie, McKenna, Erik</p>

# Module 1



## How do SNPs relate to gene transcription?

We will work with maize. 

Jupyter by default converts `tabs` into `spaces` which is good.

First, import the usual libraries

In [1]:
# basic math operations
import math 

# enable file manipulation with the OS
import os 

# more file manipulation
import glob 

# default plotter (I personally like ggplot waaaaay better. E)
import matplotlib.pyplot as plt 
%matplotlib inline # so that plots are shown in the notebook

# better plotter
import seaborn as sns 

# all the numerical cruching done here
import numpy as np 

# data wrangling
import pandas as pd 

Define some functions. Let's see for how long can we keep tidy commented code.

In [7]:
def greeting(name):
    """
    Prints a greeting for given name
    
    Parameters
    ----------
    name : string
         Name of whomever will be greeted
    
    Returns
    -------
    none : simply prints
    """
    print("Nice to have you here,", name, "!")
    
    

In [8]:
greeting("Erik")

Nice to have you here, Erik !


We can print whatever text was written between the `"""` above.

In [9]:
print(greeting.__doc__)


    Prints a greeting for given name
    
    Parameters
    ----------
    name : string
         Name of whomever will be greeted
    
    Returns
    -------
    none : simply prints
    
